In [54]:
# !pip install spacy==3.0.0

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import random
from tqdm import tqdm
from spacy.matcher import Matcher
import datetime as dt
# from spacy.tokens import Doc
# from spacy.training import Example

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/scl-2021-ds/sampleSubmission.csv
/kaggle/input/scl-2021-ds/train.csv
/kaggle/input/scl-2021-ds/test.csv


In [22]:
# df_sub = pd.read_csv("../input/scl-2021-ds/test.csv")
# submission = pd.DataFrame({
#     'id':df_sub['id'],
#     'POI/street':'/'
#     })
# submission.to_csv('7PERCENT.csv', header=True, index=False)

In [23]:
df = pd.read_csv('/kaggle/input/scl-2021-ds/train.csv')

In [24]:
df['POI'] = df['POI/street'].str.split("/", n = 2, expand = True)[0]
df['street'] = df['POI/street'].str.split("/", n = 2, expand = True)[1]

In [47]:
df.replace("", np.nan).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           300000 non-null  int64 
 1   raw_address  300000 non-null  object
 2   POI/street   300000 non-null  object
 3   POI          121491 non-null  object
 4   street       229857 non-null  object
dtypes: int64(1), object(4)
memory usage: 11.4+ MB


In [26]:
def getIndex(source, txt, label):
    start = source.find(txt)
    if start == -1:
        return start
    end = start + len(txt)
    return (start, end, label)

In [49]:
train_data = []
for x in ['POI', 'street']:
    for index, row in df.iterrows():
        li = []
        di = {}
    #     for x in ['POI', 'street']:
        if row[x] != '':
            a = getIndex(row['raw_address'], row[x], x)
            if a != -1:
                li.append(a)
                di['entities'] = li
                train_data.append((row['raw_address'], di))

In [50]:
len(train_data)

287821

```
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
    
    for _, annotation in tqdm(train_data):
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in tqdm(range(2)):
            random.shuffle(train_data)
            for text, annotations in tqdm(train_data):
                try:
                    nlp.update([text], [annotations], drop=0.2, sgd=optimizer)
                except Exception as e:
                    pass
train_model(train_data)
```

In [29]:
from spacy.util import minibatch, compounding

In [51]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("id")
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in tqdm(train_data):
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp

In [53]:
nlp = create_blank_nlp(train_data)
optimizer = nlp.begin_training()
for i in range(20):
    losses = {}
    batches = minibatch(train_data, size=compounding(3.2, 32.0, 1.0001))
    for batch in tqdm(batches):
        texts, annotations = zip(*batch)
        nlp.update(
            texts,  # batch of texts
            annotations,  # batch of annotations
            drop=0.1,  # dropout - make it harder to memorise data
            losses=losses,
        )
    print(f"Losses at iteration {i} - {dt.datetime.now()} {losses}")

100%|██████████| 287821/287821 [00:00<00:00, 518685.24it/s]
3440it [03:01, 14.00it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "pt. bank bni syariah, jl. perintis kemerdekaan km...." with entities "[(54, 62, 'POI')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "kp.pasar senen(masuk gang pesantre)rt 06 09 desa p..." with entities "[(3, 14, 'POI')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
3446it [03:01, 14.01it/s]/opt/conda/lib/python3.7/site-packages/spacy/lang

3488it [03:04, 14.06it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "jl.selang bulak rt004 002(depan perumahan casa gar..." with entities "[(26, 55, 'POI')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
3494it [03:05, 14.08it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "(warung pak ajo)terminal tanah merdeka ,rt15 12 ke..." with entities "[(16, 38, 'POI')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be

3554it [03:09, 13.99it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "kp.lebak waru des.suka senang" with entities "[(3, 13, 'POI')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
3556it [03:09, 14.00it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "metland cibitung cluster taman pluit blok r. 10 no..." with entities "[(17, 43, 'POI')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
3558it [03:09, 14.06it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be

3684it [03:18, 15.06it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "negara fitn, neg no 27" with entities "[(0, 3, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
3688it [03:18, 14.80it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "pertokoan karya ayu blok b no 6 - 9.jln dalem kaum..." with entities "[(36, 50, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
3696it [03:19, 15.22it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be a

3836it [03:28, 14.94it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "gedanganak ged 47 ungaran timur" with entities "[(0, 3, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
3844it [03:29, 15.09it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "potong ram melati, mel dalpenang" with entities "[(11, 14, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
3846it [03:29, 15.07it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in th

...

9689it [10:05, 12.66it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "ray white pesanggrahan (mulia persada amar pt), pe..." with entities "[(10, 14, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
9699it [10:06, 12.40it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "caturtunggal g 47 depok" with entities "[(8, 9, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
9703it [10:06, 11.81it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be 

9833it [10:16, 14.56it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "pondok kelapa a, 19 rt 5 12 duren sawit" with entities "[(10, 11, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
9835it [10:16, 14.45it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "muarasari muara 28 bogor selatan - kota" with entities "[(0, 5, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
9839it [10:16, 14.38it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not b

9977it [10:26, 14.57it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "sulawesi-h birdwat sula," with entities "[(0, 4, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
9999it [10:27, 14.44it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "binanga. cv, bin," with entities "[(0, 6, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
10009it [10:28, 14.50it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "wonorejo wono 1

10143it [10:38, 13.93it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "alfa wo99 ghru gaha, gah, gaharu" with entities "[(15, 18, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
10147it [10:38, 14.01it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "gedung stasiun cisauk. jl.raya cisauk lapan.sampor..." with entities "[(23, 43, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
10151it [10:39, 14.16it/s]/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities

NameError: name 'dt' is not defined

In [13]:
testdf = pd.read_csv("../input/scl-2021-ds/test.csv")

In [14]:
nlp.to_disk('nlp_model')
nlp_model = spacy.load('nlp_model')

In [15]:
testdf.head()

,id,raw_address
0,0,s. par 53 sidanegara 4 cilacap tengah
1,1,"angg per, baloi indah kel. lubuk baja"
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"


In [16]:
out = []

for _, row in tqdm(testdf.iterrows()):
    doc = nlp_model(row.raw_address)
    text = ''
    for ent in doc.ents:
        if ent.label_ == 'POI':
            text += ent.text
            text += '/'
        if ent.label_ == 'street':
            text += '/'
            text += ent.text
    if text:
        out.append(text.replace("//", "/"))
    else:
        out.append('/')

50000it [02:12, 377.98it/s]


In [17]:
submission = pd.DataFrame({'id':testdf['id'], 'POI/street': out})

In [18]:
submission.head()

,id,POI/street
0,0,/s. par
1,1,/angg per
2,2,/
3,3,/raya nga sri
4,4,/cut mutia


In [19]:
submission.to_csv('7PERCENT.csv', header=True, index=False)

In [20]:
for ent in doc.ents:
        print(f'{ent.label_.upper():{30}}- {ent.text}')

STREET                        - adi
